## Decision tree classifier

In [ ]:
import numpy as np
import pandas as pd
import warnings
import copy
import graphviz
import featureSelection
import time

from scipy.stats import mode
from sklearn.datasets import fetch_rcv1
from scipy.sparse import csc_matrix, csr_matrix, find
from scipy.stats import chi2_contingency
from sklearn.model_selection import KFold
from scipy.sparse import isspmatrix
import matplotlib.pyplot as plt

### Classifier implementation

In [ ]:
class DecisionTree():
    def __init__(self):
        self.max_depth = None
        self.max_n_nodes = None
        self.current_depth = 0
        self.idx = 0
        self.node_count = 0
        self.Y_pred = None
        self.all_nodes = []
        self.all_edges = []
        self.stop_condition = None
        self.issparse = False
        self.TPR = []
        self.FPR = []
        
    def fit(self, X, Y):
        self.node=Node(parent=None, used_features=[], depth=1, idx=0, size=X.shape[0])
        if isspmatrix(X):
            self.issparse = True
        self.rpart(X, Y, self.node)
    
    def rpart(self, X, Y, parent):
        if X.shape[1] == len(parent.used_features):
            return
        ginigain = np.zeros(X.shape[1])
        if self.issparse:
            for i in range(X.shape[1]):
                if i not in parent.used_features:
                    p = featureSelection.freq2(X[:, i], Y, True)
                    ginigain[i] = featureSelection.ginigain_fun(p)
            if np.all(np.isnan(ginigain)):
                return
            best_split = np.nanargmax(ginigain)
            ind1 = X[:, best_split].nonzero()[0]
            ind0 = np.setdiff1d(range(X.shape[0]), ind1)
            Xl = X[ind0, :]
            Xr = X[ind1, :]
            Yl = Y[ind0]
            Yr = Y[ind1]
            parent.feature = best_split
        else:
            for idx, column in enumerate(X):
                if idx not in parent.used_features:
                    xi = X[column]
                    p = featureSelection.freq2(xi, Y, True)
                    ginigain[idx] = featureSelection.ginigain_fun(p)
            best_split = np.argmax(ginigain)
            Xl = X[X.iloc[:, best_split] == True]
            Xr = X[X.iloc[:, best_split] == False]
            Yl = Y[X.iloc[:, best_split] == True]
            Yr = Y[X.iloc[:, best_split] == False]
            parent.feature = Xl.columns[best_split]

        parent.pi = featureSelection.freq(Y, True)
        used_features = copy.copy(parent.used_features)
        used_features.append(best_split)  

        self.idx += 1
        parent.left = Node(parent, used_features, parent.depth + 1, self.idx, Xl.shape[0])

        self.idx += 1
        parent.right = Node(parent, used_features, parent.depth + 1, self.idx, Xr.shape[0])
        if not self.StopCondition(Y, Yr, parent.depth + 1):
            self.rpart(Xr, Yr, parent.right)
        else:
            if self.issparse:
                if Yr.nonzero()[0].shape[0] / Xr.shape[0] >= 0.4:
                    parent.right.leaf_val = 1
                else:
                    parent.right.leaf_val = 0
            else:
                parent.right.leaf_val = mode(Yr)[0]
        if not self.StopCondition(Y, Yl, parent.depth + 1):
            self.rpart(Xl, Yl, parent.left)
        else:
            if self.issparse:
                if Yl.nonzero()[0].shape[0] / Xl.shape[0] >= 0.5:
                    parent.left.leaf_val = 1
                else:
                    parent.left.leaf_val = 0
            else:
                parent.left.leaf_val = mode(Yl)[0]
                                                
    def StopCondition(self, Y, Y_child, depth):
        if self.issparse:
            if self.stop_condition == 'depth':
                if depth >= self.max_depth:
                    return True
            else:
                if self.idx >= self.max_n_nodes:
                    return True
            return False
        name_Y, pi_Y = featureSelection.freq(Y, True)
        name_Y_child, pi_Y_child = featureSelection.freq(Y_child, True)
        if self.stop_condition == 'depth':
            if depth == self.max_depth or len(name_Y_child) <= 1 :
                return True
        elif self.stop_condition == 'n_nodes':
            if self.max_n_nodes >= self.idx or len(name_Y_child) <= 1 :
                return True
        else:
            diff = np.setdiff1d(name_Y, name_Y_child)
            pi_Y_child_whole = np.zeros_like(pi_Y)
            j = 0
            for i, name in enumerate(name_Y):
                if name not in diff:
                    pi_Y_child_whole[i] = pi_Y_child[j]
                    j += 1
            crosstab = pd.crosstab(pi_Y, pi_Y_child_whole)
            chi2, p_value, degrees_of_freedom, expected_frequencies = chi2_contingency(crosstab)
            if p_value > 0.3 or len(name_Y_child) <= 1:
                return True
        return False

    def predict(self, X):
        if self.issparse:
            self.Y_pred = []
            for xi in X:
                self.ppart_sparse(xi, self.node)
        else:
            self.Y_pred = np.zeros(X.shape[0], dtype=object)
            self.ppart(X, self.node, X)
        
    def ppart_sparse(self, x, node):
        if node.leaf_val != None:
            self.Y_pred.append(node.leaf_val)
        else:
            if x[:, node.feature] == 1:
                self.ppart_sparse(x, node.right)
            else:
                self.ppart_sparse(x, node.left)
                
    def ppart(self, X, node, mainX):
        if node.leaf_val != None:
            idx_X = [] 
            for idx, row_mainX in enumerate(mainX.T):
                for rowX in X.T:
                    if rowX == row_mainX:
                        idx_X.append(idx)
            for i in idx_X:
                self.Y_pred[i] = node.leaf_val[0]
        else:
            Xl = X[X.loc[:, node.feature] == True]
            Xr = X[X.loc[:, node.feature] == False]
            self.ppart(Xl, node.left, mainX)
            self.ppart(Xr, node.right, mainX)
                
    def score(self, Y):
        if self.issparse:
            equal = np.ravel(Y.toarray())== np.asarray(self.Y_pred)
        else:
            equal = Y == self.Y_pred
        unique, counts = np.unique(equal, return_counts=True)
        if len(unique) == 2:
            if unique[0] == False:
                return counts[0]/len(self.Y_pred)
            else:
                return counts[1]/len(self.Y_pred)
        else:
            if unique == True:
                return 0
        return 1

    def graph(self):
        self.all_nodes = []
        self.all_edges = []
        dot = graphviz.Digraph(comment='Decision tree')
        if self.issparse:
            self.gpart_sparse(self.node, None, words)
        else:
            self.gpart(self.node, None)
        for i in self.all_nodes:
            dot.node(str(i[0]), str(i[1]), shape='box')
        for i in self.all_edges:
            dot.edge(str(i[1]), str(i[0]))
        return dot
    
    def gpart(self, node, parent):
        node_data = ''
        if node.feature != None:
            node_data += 'feature:'+node.feature+'\n'
        if node.pi != None:
            node_data += 'pi: '+''.join(np.array2string(np.around(node.pi[1], decimals=2)))+'\n'
        if node.leaf_val != None:
            node_data += 'class: '+str(node.leaf_val[0])+'\n'
        node_data += 'samples = '+str(node.size)
        self.all_nodes.append([node.idx, node_data])
        if parent != None:
            self.all_edges.append([node.idx, parent])
        if node.leaf_val == None:
            self.gpart(node.left, node.idx)
            self.gpart(node.right, node.idx)
            
    def gpart_sparse(self, node, parent, words):
        node_data = ''
        node_data += 'idx='+str(node.idx)+'\n samples = '+str(node.size)
        if node.feature != None:
            node_data += '\n word: '+words[node.feature]
        if node.leaf_val != None:
            node_data += '\n dec= '+str(node.leaf_val)
        self.all_nodes.append([node.idx, node_data])
        if parent != None:
            self.all_edges.append([node.idx, parent])
        if node.leaf_val == None:
            self.gpart_sparse(node.left, node.idx, words)
            self.gpart_sparse(node.right, node.idx, words)
            
    def get_params(self, deep=True): 
        return {"stop_condition": self.stop_condition, "max_depth": self.max_depth, "max_n_nodes": self.max_n_nodes}

    def set_params(self, parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
                
    def crossvalidation(self, X, Y, depth, n_splits, max_n_nodes, stop_condition):
        score = []
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=4)
        for train_index, test_index in kf.split(X):
            self.set_params({"max_depth": depth, "max_n_nodes": max_n_nodes, "stop_condition": stop_condition})
            if isspmatrix(X):
                X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
                self.fit(X_train.tocsc(), y_train.tocsc())
            else:
                X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], \
                                                Y.iloc[train_index], Y.iloc[test_index]
                self.fit(X_train, y_train)
            self.predict(X_test)
            score.append(self.score(y_test))
        return score, y_test, self.Y_pred
           
    def roc(self):
        pass
    
    def confusion_matrix(self, y_test, y_pred):
        y_test_ = np.ravel(y_test.toarray())
        y_pred_ = np.asarray(d.Y_pred)
        unique, counts = np.unique(y_test_*10+y_pred_, return_counts=True)
        TN = 0
        FN = 0
        FP = 0 
        TP = 0
        for ui, ci in zip(unique, counts):
            if ui == 0:
                TN = ci
            elif ui == 1:
                FP = ci
            elif ui == 10:
                FN = ci
            else:
                TP = ci
        return TN, FN, FP, TP

    def TPR_FPR(self, y_test, y_pred):
        TN, FN, FP, TP = self.confusion_matrix(y_test, y_pred)
        self.TPR.append(TP/(TP+FN))
        self.FPR.append(FP/(FP+TN))
        
class Node():
    def __init__(self, parent, used_features, depth, idx, size):
        self.feature = None
        self.used_features = used_features
        self.pi = None
        self.left=None
        self.right=None
        self.depth=depth
        self.idx = idx
        self.leaf_val = None
        self.size = size

### Loading zoo dataset, classification, tree visualization

In [ ]:
zoo_df = pd.read_csv('zoo.csv')
zoo_df = zoo_df.drop(['animal'], axis=1)
kappa_zoo = []
X = zoo_df.drop(['type'], axis=1)
X['legs'] = X['legs'].apply(lambda x: x == 4)
Y = zoo_df['type']

In [ ]:
import time
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

d = DecisionTree()
d.set_params({"max_depth": 7, "max_n_nodes":6, "stop_condition":"depth"})
start_time = time.time()
d.fit(X_train, y_train)
print("FIT TIME:", time.time() - start_time, "s")

start_time = time.time()
d.predict(X_test)
print("PREDICT TIME:", time.time() - start_time, "s")
print("ERROR:", d.score(y_test))

dot = d.graph()
dot.format = 'png'
dot.view(filename='zoo', directory='./data/')

### Crossvalidation (own implementation)

In [ ]:
start = time.time()
d = DecisionTree()

d.crossvalidation(X, Y, 8, 4, 30, "n_nodes")
print("TIME:", time.time() - start)

### Crossvalidation (Sklearn)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

start = time.time()
clf = DecisionTreeClassifier()
gc = GridSearchCV(clf, {"max_depth": [8]},cv=4, verbose=0).fit(X, Y)
print("TIME:", time.time() - start)

### Loading RCV1 dataset, classification, tree visualization

In [ ]:
rcv1 = fetch_rcv1()
X = rcv1["data"]
y = rcv1.target[:, 87]
X = X>0

In [ ]:
words = []
with open('stem.termid.idf.map.txt', "r", encoding="utf8") as reader:
    for line in reader:
        words.append(line.split()[0])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train = X_train.tocsc()
y_train = y_train.tocsc()

In [ ]:
d = DecisionTree()
d.set_params({"max_depth": 5, "max_n_nodes":2, "stop_condition":"depth"})
start_time = time.time()
d.fit(X_train, y_train)
print("FIT TIME:", time.time() - start_time, "s")

start_time = time.time()
d.predict(X_test)
print("PREDICT TIME:", time.time() - start_time, "s")
print("ERROR:", d.score(y_test))

dot = d.graph()
dot.format = 'png'
dot.view(filename='rcv1', directory='./')

### Crossvalidation (own implementation)

In [ ]:
warnings.filterwarnings('ignore')
d = DecisionTree()
start = time.time()
d.crossvalidation(X, y, 3, 5, 3, "depth")
print("TIME:", time.time() - start)

### Crossvalidation (Sklearn)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

start = time.time()
clf = DecisionTreeClassifier()
gc = GridSearchCV(clf, {"max_depth": [3]}, cv=5, verbose=0).fit(X, y.todense())
print("TIME:", time.time() - start)

### ROC curve

In [ ]:
d = DecisionTree()
for i in range(2, 9):
    d.set_params({"max_depth": i, "max_n_nodes":2, "stop_condition":"depth"})
    start_time = time.time()
    d.fit(X_train, y_train)
    print("FIT TIME:", time.time() - start_time, "s")

    start_time = time.time()
    d.predict(X_test)
    print("PREDICT TIME:", time.time() - start_time, "s")
    print("ERROR:", d.score(y_test))

    d.TPR_FPR(y_test, d.Y_pred)

In [ ]:
plt.plot(d.FPR, d.TPR)
plt.show()